# All oscillation analysis

Brief 1-2 sentence description of notebook.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os
import collections
import itertools
from collections import defaultdict
from itertools import combinations

In [3]:
os.environ["SPECTRAL_CONNECTIVITY_ENABLE_GPU"] = "true"
import cupy as cp

In [4]:
# Imports of all used packages and libraries
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import mannwhitneyu


In [5]:
import matplotlib
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import colorsys

In [6]:
from sklearn.metrics import confusion_matrix

In [7]:
from spectral_connectivity import Multitaper, Connectivity
import spectral_connectivity

In [8]:
FONTSIZE = 20

In [9]:
font = {'weight' : 'medium',
        'size'   : 20}

matplotlib.rc('font', **font)

# Functions

In [10]:
def generate_pairs(lst):
    """
    Generates all unique pairs from a list.

    Parameters:
    - lst (list): The list to generate pairs from.

    Returns:
    - list: A list of tuples, each containing a unique pair from the input list.
    """
    n = len(lst)
    return [(lst[i], lst[j]) for i in range(n) for j in range(i+1, n)]

In [11]:
def update_array_by_mask(array, mask, value=np.nan):
    """
    """
    result = array.copy()
    array[mask] = value
    return array

## Inputs & Data

Explanation of each input and where it comes from.

In [12]:
# Inputs and Required data loading
# input varaible names are in all caps snake case
# Whenever an input changes or is used for processing 
# the vairables are all lower in snake case
OUTPUT_DIR = r"./proc" # where data is saved should always be shown in the inputs

In [13]:
TIME_HALFBANDWIDTH_PRODUCT = 2
TIME_WINDOW_DURATION = 1
TIME_WINDOW_STEP = 0.5
RESAMPLE_RATE=1000

In [14]:
zscore_threshold = 3

In [15]:
BAND_TO_FREQ = {"theta": (6,11), "beta": (20,31), "gamma": (30,51)}

In [16]:
LFP_TRACES_DF = pd.read_pickle("./proc/rce_pilot_2_01_lfp_traces_and_frames.pkl")

In [17]:
LFP_TRACES_DF.shape

(61, 23)

## Preprocessing

In [18]:
original_trace_columns = [col for col in LFP_TRACES_DF.columns if "trace" in col]

In [19]:
original_trace_columns

['mPFC_lfp_trace',
 'MD_lfp_trace',
 'LH_lfp_trace',
 'BLA_lfp_trace',
 'vHPC_lfp_trace']

In [20]:
for col in original_trace_columns:
    print(col)
    LFP_TRACES_DF[col] = LFP_TRACES_DF[col].apply(lambda x: x.astype(np.float32))
    # LFP_TRACES_DF[col] = LFP_TRACES_DF[col].apply(lambda x: x.astype(int))

mPFC_lfp_trace
MD_lfp_trace
LH_lfp_trace
BLA_lfp_trace
vHPC_lfp_trace


In [21]:
LFP_TRACES_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,video_timestamps,tone_timestamps,box_1_port_entry_timestamps,box_2_port_entry_timestamps,lfp_timestamps,mPFC_lfp_trace,MD_lfp_trace,LH_lfp_trace,BLA_lfp_trace,vHPC_lfp_trace
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[489.0, 421.0, 500.0, 679.0, 635.0, 474.0, 382...","[236.0, 253.0, 386.0, 502.0, 398.0, 208.0, 115...","[315.0, 339.0, 420.0, 464.0, 367.0, 208.0, 133...","[280.0, 279.0, 376.0, 442.0, 303.0, 102.0, -13...","[285.0, 407.0, 657.0, 874.0, 971.0, 959.0, 936..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[489.0, 421.0, 500.0, 679.0, 635.0, 474.0, 382...","[236.0, 253.0, 386.0, 502.0, 398.0, 208.0, 115...","[315.0, 339.0, 420.0, 464.0, 367.0, 208.0, 133...","[280.0, 279.0, 376.0, 442.0, 303.0, 102.0, -13...","[285.0, 407.0, 657.0, 874.0, 971.0, 959.0, 936..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[102.0, 151.0, 170.0, 194.0, 233.0, 90.0, -173...","[151.0, 148.0, 133.0, 119.0, 110.0, 64.0, 4.0,...","[146.0, 130.0, 113.0, 117.0, 118.0, 71.0, 3.0,...","[352.0, 463.0, 481.0, 367.0, 462.0, 639.0, 671...","[323.0, 445.0, 534.0, 446.0, 388.0, 465.0, 567..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[-2, 1384, 2770, 4156, 4156, 5542, 6928, 6928,...","[[982229, 1182226], [3382227, 3582224], [56822...","[[491029, 515227], [519426, 558629], [559427, ...","[[33082200, 33090003], [33565003, 33567000], [...","[0, 20, 40, 60, 80, 100, 120, 140, 160, 180, 2...","[102.0, 151.0, 170.0, 194.0, 233.0, 90.0, -173...","[151.0, 148.0, 133.0, 119.0, 110.0, 64.0, 4.0,...","[146.0, 130.0, 113.0, 117.0, 118.0, 71.0, 3.0,...","[352.0, 463.0, 481.0, 367.0, 462.0, 639.0, 671...","[323.0, 445.0, 534.0, 446.0, 388.0, 465.0, 567..."
4,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 24

In [22]:
LFP_TRACES_DF["mPFC_lfp_trace"].apply(lambda x: np.abs(x).mean()).head(n=20)

0     846.054077
1     846.054077
2     300.934784
3     300.934784
4     413.799805
5     413.799805
6     382.395569
7     382.395569
8     419.452911
9     419.452911
10    265.210266
11    265.210266
12    412.869934
13    412.869934
14    412.869934
15    386.954834
16    386.954834
17    386.954834
18    400.192047
19    400.192047
Name: mPFC_lfp_trace, dtype: float32

## Calculating zscore

In [23]:
for col in original_trace_columns:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_lfp_zscore".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: stats.zscore(x).astype(np.float32))

mPFC_lfp_trace
MD_lfp_trace
LH_lfp_trace
BLA_lfp_trace
vHPC_lfp_trace


In [24]:
LFP_TRACES_DF[updated_column]

0     [0.08100804, 0.12151435, 0.20451908, 0.2765672...
1     [0.08100804, 0.12151435, 0.20451908, 0.2765672...
2     [0.8864209, 1.221237, 1.4654881, 1.2239814, 1....
3     [0.8864209, 1.221237, 1.4654881, 1.2239814, 1....
4     [0.26165456, 0.38504797, 0.3374746, -0.0104055...
                            ...                        
56    [0.21208751, 0.7588723, 1.1022531, 0.6845096, ...
57    [0.17408794, 0.0098705115, -0.14996777, -0.199...
58    [0.17408794, 0.0098705115, -0.14996777, -0.199...
59    [-1.0134448, -1.1096387, -1.0760826, -0.861324...
60    [-1.0134448, -1.1096387, -1.0760826, -0.861324...
Name: vHPC_lfp_zscore, Length: 61, dtype: object

## calculating root mean sequare

In [25]:
for col in original_trace_columns:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_lfp_RMS".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: (x / np.sqrt(np.mean(x**2))).astype(np.float32))


mPFC_lfp_trace
MD_lfp_trace
LH_lfp_trace
BLA_lfp_trace
vHPC_lfp_trace


In [26]:
LFP_TRACES_DF.head()

,cohort,session_dir,tone_frames,box_1_port_entry_frames,box_2_port_entry_frames,video_name,session_path,recording,current_subject,subject,...,mPFC_lfp_zscore,MD_lfp_zscore,LH_lfp_zscore,BLA_lfp_zscore,vHPC_lfp_zscore,mPFC_lfp_RMS,MD_lfp_RMS,LH_lfp_RMS,BLA_lfp_RMS,vHPC_lfp_RMS
0,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[0.19068325, 0.16359733, 0.19506478, 0.2663644...","[0.13075392, 0.14042763, 0.21611021, 0.2821191...","[0.22135854, 0.2382851, 0.29541218, 0.32644418...","[0.15432237, 0.15375327, 0.20895602, 0.2465166...","[0.08100804, 0.12151435, 0.20451908, 0.2765672...","[0.19477786, 0.16769218, 0.19915937, 0.2704584...","[0.13429303, 0.14396669, 0.21964878, 0.2856572...","[0.22216085, 0.23908739, 0.29621446, 0.3272464...","[0.15934612, 0.15877703, 0.21397908, 0.2515392...","[0.09461664, 0.1351192, 0.21811624, 0.29015768..."
1,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.3,1.3,...,"[0.19068325, 0.16359733, 0.19506478, 0.2663644...","[0.13075392, 0.14042763, 0.21611021, 0.2821191...","[0.22135854, 0.2382851, 0.29541218, 0.32644418...","[0.15432237, 0.15375327, 0.20895602, 0.2465166...","[0.08100804, 0.12151435, 0.20451908, 0.2765672...","[0.19477786, 0.16769218, 0.19915937, 0.2704584...","[0.13429303, 0.14396669, 0.21964878, 0.2856572...","[0.22216085, 0.23908739, 0.29621446, 0.3272464...","[0.15934612, 0.15877703, 0.21397908, 0.2515392...","[0.09461664, 0.1351192, 0.21811624, 0.29015768..."
2,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1181], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33137, 33147], [33665, 33666], [33668, 33669...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[0.2584528, 0.38262582, 0.43077454, 0.491594, ...","[0.699254, 0.68536323, 0.61590964, 0.5510862, ...","[0.6526404, 0.58112735, 0.50514466, 0.5230229,...","[0.8357042, 1.0992358, 1.1419706, 0.8713166, 1...","[0.8864209, 1.221237, 1.4654881, 1.2239814, 1....","[0.25848264, 0.38265565, 0.43080437, 0.4916238...","[0.6991664, 0.68527573, 0.6158221, 0.5509987, ...","[0.6525567, 0.58104366, 0.50506103, 0.5229393,...","[0.83570397, 1.0992357, 1.1419705, 0.8713164, ...","[0.8864393, 1.2212554, 1.4655064, 1.2239997, 1..."
3,2,20230612_101430_standard_comp_to_training_D1_s...,"[[980, 1180], [3376, 3575], [5672, 5871], [746...","[[490, 514], [518, 558], [558, 637], [638, 640...","[[33021, 33027], [33502, 33503], [33504, 33506...",20230612_101430_standard_comp_to_training_D1_s...,/scratch/back_up/reward_competition_extention/...,20230612_101430_standard_comp_to_training_D1_s...,1.4,1.4,...,"[0.2584528, 0.38262582, 0.43077454, 0.491594, ...","[0.699254, 0.68536323, 0.61590964, 0.5510862, ...","[0.6526404, 0.58112735, 0.50514466, 0.5230229,...","[0.8357042, 1.0992358, 1.1419706, 0.8713166, 1...","[0.8864209, 1.221237, 1.4654881, 1.2239814, 1....","[0.25848264, 0.38265565, 0.43080437, 0.4916238...","[0.6991664, 0.68527573, 0.6158221, 0.5509987, ...","[0.6525567, 0.58104366, 0.50506103, 0.5229393,...","[0.83570397, 1.0992357, 1.1419705, 0.8713164, ...","[0.8864393, 1.2212554, 1.4655064, 1.2239997, 1..."
4,2,20230612_112630_standard_comp_to_training_D1_s...,"[[1125, 1324], [3519, 3720], [5815, 6014], [76...","[[192, 248], [389, 405], [916, 929], [929, 948..

## Filtering for zscore value

In [27]:
zscore_columns = [col for col in LFP_TRACES_DF.columns if "zscore" in col]

In [28]:
zscore_columns

['mPFC_lfp_zscore',
 'MD_lfp_zscore',
 'LH_lfp_zscore',
 'BLA_lfp_zscore',
 'vHPC_lfp_zscore']

In [29]:
for col in zscore_columns:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_lfp_mask".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: np.abs(x) >= zscore_threshold)

mPFC_lfp_zscore
MD_lfp_zscore
LH_lfp_zscore
BLA_lfp_zscore
vHPC_lfp_zscore


In [30]:
LFP_TRACES_DF[updated_column].head()

0    [False, False, False, False, False, False, Fal...
1    [False, False, False, False, False, False, Fal...
2    [False, False, False, False, False, False, Fal...
3    [False, False, False, False, False, False, Fal...
4    [False, False, False, False, False, False, Fal...
Name: vHPC_lfp_mask, dtype: object

In [31]:
LFP_TRACES_DF[updated_column].iloc[0].shape

(3414668,)

In [32]:
sum(LFP_TRACES_DF[updated_column].iloc[0])

70398

- Filtering raw traces by zscore

In [33]:
LFP_TRACES_DF[col].head()

0    [0.08100804, 0.12151435, 0.20451908, 0.2765672...
1    [0.08100804, 0.12151435, 0.20451908, 0.2765672...
2    [0.8864209, 1.221237, 1.4654881, 1.2239814, 1....
3    [0.8864209, 1.221237, 1.4654881, 1.2239814, 1....
4    [0.26165456, 0.38504797, 0.3374746, -0.0104055...
Name: vHPC_lfp_zscore, dtype: object

In [34]:
for col in original_trace_columns:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_lfp_trace_filtered".format(brain_region)    
    mask_column = "{}_lfp_mask".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF.apply(lambda x: update_array_by_mask(x[col], x[mask_column]), axis=1)

mPFC_lfp_trace
MD_lfp_trace
LH_lfp_trace
BLA_lfp_trace
vHPC_lfp_trace


In [35]:
LFP_TRACES_DF[col].head()

0    [285.0, 407.0, 657.0, 874.0, 971.0, 959.0, 936...
1    [285.0, 407.0, 657.0, 874.0, 971.0, 959.0, 936...
2    [323.0, 445.0, 534.0, 446.0, 388.0, 465.0, 567...
3    [323.0, 445.0, 534.0, 446.0, 388.0, 465.0, 567...
4    [176.0, 259.0, 227.0, -7.0, -163.0, -145.0, -3...
Name: vHPC_lfp_trace, dtype: object

In [36]:
sum(np.isnan(LFP_TRACES_DF[col].iloc[0]))

70398

- Calculating RMS of filtered signal

In [37]:
filtered_trace_column = [col for col in LFP_TRACES_DF if "lfp_trace_filtered" in col]

In [38]:
for col in filtered_trace_column:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_lfp_RMS_filtered".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: (x / np.sqrt(np.nanmean(x**2))).astype(np.float32))

mPFC_lfp_trace_filtered
MD_lfp_trace_filtered
LH_lfp_trace_filtered
BLA_lfp_trace_filtered
vHPC_lfp_trace_filtered


## Calculating phase of signals

In [39]:
from scipy.signal import butter, filtfilt, hilbert

- Filtering for theta and gamma

In [53]:
RMS_columns = [col for col in LFP_TRACES_DF if "RMS" in col and "filtered" not in col]

In [54]:
fs = 1000
order=4

In [55]:
freq_band = [4, 12]
b, a = butter(order, freq_band, fs=fs, btype='band')

In [56]:
for col in RMS_columns:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_theta_band".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: filtfilt(b, a, x, padtype=None).astype(np.float32))

mPFC_lfp_RMS
MD_lfp_RMS
LH_lfp_RMS
BLA_lfp_RMS
vHPC_lfp_RMS


In [57]:
freq_band = [30, 50]
b, a = butter(order, freq_band, fs=fs, btype='band')

In [58]:
for col in RMS_columns:
    print(col)
    brain_region = col.split("_")[0]
    updated_column = "{}_gamma_band".format(brain_region)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: filtfilt(b, a, x, padtype=None).astype(np.float32))

mPFC_lfp_RMS
MD_lfp_RMS
LH_lfp_RMS
BLA_lfp_RMS
vHPC_lfp_RMS


- Calculating the phase

In [59]:
band_columns = [col for col in LFP_TRACES_DF if "band" in col]

In [60]:
band_columns

['mPFC_theta_band',
 'MD_theta_band',
 'LH_theta_band',
 'BLA_theta_band',
 'vHPC_theta_band',
 'mPFC_gamma_band',
 'MD_gamma_band',
 'LH_gamma_band',
 'BLA_gamma_band',
 'vHPC_gamma_band']

In [61]:
for col in band_columns:
    print(col)
    brain_region = col.replace("_band", "")
    updated_column = "{}_phase".format(brain_region)
    print(updated_column)
    LFP_TRACES_DF[updated_column] = LFP_TRACES_DF[col].apply(lambda x: np.angle(hilbert(x), deg=False))

mPFC_theta_band
mPFC_theta_phase


KeyboardInterrupt: 

In [62]:
LFP_TRACES_DF[col]

0     [0.060033023, 0.064530484, 0.068970464, 0.0733...
1     [0.060033023, 0.064530484, 0.068970464, 0.0733...
2     [-0.46605772, -0.50434494, -0.5420351, -0.5789...
3     [-0.46605772, -0.50434494, -0.5420351, -0.5789...
4     [-0.07290424, -0.07259816, -0.072436996, -0.07...
                            ...                        
56    [0.093562424, 0.124071084, 0.15454663, 0.18488...
57    [0.5179356, 0.5202326, 0.5217201, 0.52236974, ...
58    [0.5179356, 0.5202326, 0.5217201, 0.52236974, ...
59    [0.18128568, 0.15002851, 0.118328236, 0.086277...
60    [0.18128568, 0.15002851, 0.118328236, 0.086277...
Name: mPFC_theta_band, Length: 61, dtype: object

In [ ]:
raise ValueError()

# Power Calcuation

- Getting the column name of all the traces

In [ ]:
input_columns = [col for col in LFP_TRACES_DF.columns if "trace" in col or "RMS" in col]

In [ ]:
input_columns

In [ ]:
for col in input_columns:
    print(col)
    LFP_TRACES_DF[col] = LFP_TRACES_DF[col].apply(lambda x: x.astype(np.float32))

- Calcuating the power at each frequency band

In [ ]:
for col in input_columns:
    # brain_region = col.split("_")[0]
    brain_region = col.replace("_lfp", "")
    print(brain_region)

    # Define column names
    multitaper_col = f"{brain_region}_power_multitaper"
    connectivity_col = f"{brain_region}_power_connectivity"
    frequencies_col = f"{brain_region}_power_frequencies"
    power_col = f"{brain_region}_power_all_frequencies_all_windows"
    
    try:
        # Apply Multitaper function to the lfp_trace column
        LFP_TRACES_DF[multitaper_col] = LFP_TRACES_DF[col].apply(
            lambda x: Multitaper(
                time_series=x, 
                sampling_frequency=RESAMPLE_RATE, 
                time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT,
                time_window_duration=TIME_WINDOW_DURATION, 
                time_window_step=TIME_WINDOW_STEP
            )
        )

        # Apply Connectivity function to the multitaper column
        LFP_TRACES_DF[connectivity_col] = LFP_TRACES_DF[multitaper_col].apply(
            lambda x: Connectivity.from_multitaper(x)
        )

        # Apply frequencies and power functions to the connectivity column
        LFP_TRACES_DF[frequencies_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.frequencies
        )
        LFP_TRACES_DF[power_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.power().squeeze()
        )
        
        LFP_TRACES_DF[power_col] = LFP_TRACES_DF[power_col].apply(lambda x: x.astype(np.float16))
            
        # Removing unnecessary columns
        LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[multitaper_col, connectivity_col], errors="ignore")
    
    except Exception as e: 
        print(e)

- Getting the timestamps of the power

In [ ]:
LFP_TRACES_DF["power_timestamps"] = LFP_TRACES_DF["lfp_timestamps"].apply(lambda x: x[(RESAMPLE_RATE//2):(-RESAMPLE_RATE//2):(RESAMPLE_RATE//2)])
# .iloc[0][500:-500:500].shape

- Making sure that the timestamps for power makes sense with shape and values

In [ ]:
LFP_TRACES_DF["power_timestamps"].head().apply(lambda x: x.shape)

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_all_frequencies_all_windows" in col][0]].iloc[0].shape

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "lfp_timestamps" in col][0]].iloc[0]

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_timestamps" in col][0]].iloc[0]

- Checking if the right frequencies are being used

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_frequencies" in col]].head()

In [ ]:
LFP_TRACES_DF["power_calculation_frequencies"] = LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "power_frequencies" in col][0]].copy()

- Dropping unnecessary columns

In [ ]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "power_frequencies" in col], errors="ignore")

In [ ]:
LFP_TRACES_DF.head()

In [ ]:
LFP_TRACES_DF["mPFC_RMS_filtered_power_all_frequencies_all_windows"].head()

In [ ]:
LFP_TRACES_DF["mPFC_RMS_filtered_power_all_frequencies_all_windows"].iloc[4].shape

In [ ]:
LFP_TRACES_DF["mPFC_lfp_RMS_filtered"].head()

In [ ]:
plt.plot(LFP_TRACES_DF["BLA_lfp_trace"].iloc[0][:100000])

In [ ]:
plt.plot(LFP_TRACES_DF["BLA_lfp_RMS"].iloc[0][:100000])

In [ ]:
plt.plot(LFP_TRACES_DF["BLA_lfp_RMS_filtered"].iloc[0][:100000])

In [ ]:
LFP_TRACES_DF["BLA_trace_power_all_frequencies_all_windows"].apply(lambda x: np.sum(np.isnan(x[:,3:13])))

In [ ]:
LFP_TRACES_DF["BLA_RMS_filtered_power_all_frequencies_all_windows"].apply(lambda x: np.sum(np.isnan(x[:,3:13])))

In [ ]:
raise ValueError()

In [ ]:
# LFP_TRACES_DF.to_pickle("./proc/rce2_spectral_granger.pkl")
LFP_TRACES_DF.to_pickle("./proc/rce_pilot_2_02_full_spectral.pkl")
# LFP_TRACES_DF.to_pickle("/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/rce_pilot_2_02_spectral_granger.pkl")

In [ ]:
raise ValueError()

## Coherence Calculation

- Getting the trace column pairs

In [ ]:
trace_columns

In [ ]:
brain_region_pairs = generate_pairs(sorted(trace_columns))
brain_region_pairs = sorted(brain_region_pairs)


In [ ]:
brain_region_pairs

## Coherece Calculation

- Calculating the coherence

In [ ]:
for region_1, region_2 in brain_region_pairs:
    # Define base name for pair
    pair_base_name = f"{region_1.split('_')[0]}_{region_2.split('_')[0]}"
    print(pair_base_name)

    try:
        # Define column names
        multitaper_col = f"{pair_base_name}_coherence_multitaper"
        connectivity_col = f"{pair_base_name}_coherence_connectivity"
        frequencies_col = f"{pair_base_name}_coherence_frequencies"
        coherence_col = f"{pair_base_name}_coherence_all_frequencies_all_windows"

        # Apply Multitaper function
        LFP_TRACES_DF[multitaper_col] = LFP_TRACES_DF.apply(
            lambda x: Multitaper(
                time_series=np.array([x[region_1], x[region_2]]).T, 
                sampling_frequency=RESAMPLE_RATE, 
                time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT, 
                time_window_step=TIME_WINDOW_STEP, 
                time_window_duration=TIME_WINDOW_DURATION
            ), 
            axis=1
        )

        # Apply Connectivity function
        LFP_TRACES_DF[connectivity_col] = LFP_TRACES_DF[multitaper_col].apply(
            lambda x: Connectivity.from_multitaper(x)
        )

        # Apply frequencies and coherence functions
        LFP_TRACES_DF[frequencies_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.frequencies
        )
        LFP_TRACES_DF[coherence_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.coherence_magnitude()[:,:,0,1]
        )

        LFP_TRACES_DF[coherence_col] = LFP_TRACES_DF[coherence_col].apply(lambda x: x.astype(np.float16))

    except Exception as e: 
        print(e)

    # Drop temporary columns
    LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[multitaper_col, connectivity_col], errors="ignore")

- Getting the timestamps of the coherence

In [ ]:
LFP_TRACES_DF["coherence_timestamps"] = LFP_TRACES_DF["lfp_timestamps"].apply(lambda x: x[(RESAMPLE_RATE//2):(-RESAMPLE_RATE//2):(RESAMPLE_RATE//2)])


- Making sure that the timestamps for coherence makes sense with shape and values

In [ ]:
LFP_TRACES_DF["coherence_timestamps"].head().apply(lambda x: x.shape)

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_all_frequencies_all_windows" in col][0]].iloc[0].shape

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "lfp_timestamps" in col][0]].iloc[0]

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_timestamps" in col][0]].iloc[0]

- Checking if the right frequencies are being used

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_frequencies" in col]].head()

In [ ]:
LFP_TRACES_DF["coherence_calculation_frequencies"] = LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "coherence_frequencies" in col][0]].copy()

- Dropping unnecessary columns

In [ ]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "coherence_frequencies" in col], errors="ignore")

In [ ]:
LFP_TRACES_DF.head()

In [ ]:
LFP_TRACES_DF.to_pickle("./proc/rce2_spectral_coherence.pkl")

# Calculate Granger's

In [ ]:
for region_1, region_2 in brain_region_pairs:
    # Define base name for pair
    region_1_base_name = region_1.split('_')[0]
    region_2_base_name = region_2.split('_')[0]

    pair_base_name = f"{region_1_base_name}_{region_2_base_name}"
    print(pair_base_name)

    try:
        # Define column names
        multitaper_col = f"{pair_base_name}_granger_multitaper"
        connectivity_col = f"{pair_base_name}_granger_connectivity"
        frequencies_col = f"{pair_base_name}_granger_frequencies"
        granger_1_2_col = f"{region_1_base_name}_{region_2_base_name}_granger_all_frequencies_all_windows"
        granger_2_1_col = f"{region_2_base_name}_{region_1_base_name}_granger_all_frequencies_all_windows"

        # Apply Multitaper function
        LFP_TRACES_DF[multitaper_col] = LFP_TRACES_DF.apply(
            lambda x: Multitaper(
                time_series=np.array([x[region_1], x[region_2]]).T, 
                sampling_frequency=RESAMPLE_RATE, 
                time_halfbandwidth_product=TIME_HALFBANDWIDTH_PRODUCT, 
                time_window_step=TIME_WINDOW_STEP, 
                time_window_duration=TIME_WINDOW_DURATION
            ), 
            axis=1
        )
    
        # Apply Connectivity function
        LFP_TRACES_DF[connectivity_col] = LFP_TRACES_DF[multitaper_col].apply(
            lambda x: Connectivity.from_multitaper(x)
        )

        # Apply frequencies and granger functions
        LFP_TRACES_DF[frequencies_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.frequencies
        )
        
        LFP_TRACES_DF[granger_1_2_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.pairwise_spectral_granger_prediction()[:,:,0,1]
        )

        LFP_TRACES_DF[granger_2_1_col] = LFP_TRACES_DF[connectivity_col].apply(
            lambda x: x.pairwise_spectral_granger_prediction()[:,:,1,0]
        )

        LFP_TRACES_DF[granger_1_2_col] = LFP_TRACES_DF[granger_1_2_col].apply(lambda x: x.astype(np.float16))
        LFP_TRACES_DF[granger_2_1_col] = LFP_TRACES_DF[granger_2_1_col].apply(lambda x: x.astype(np.float16))
        
    except Exception as e: 
        print(e)

    # Drop temporary columns
    LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[multitaper_col, connectivity_col], errors="ignore")

- Getting the timestamps of the granger

In [ ]:
LFP_TRACES_DF["granger_timestamps"] = LFP_TRACES_DF["lfp_timestamps"].apply(lambda x: x[(RESAMPLE_RATE//2):(-RESAMPLE_RATE//2):(RESAMPLE_RATE//2)])


- Making sure that the timestamps for granger makes sense with shape and values

In [ ]:
LFP_TRACES_DF["granger_timestamps"].head().apply(lambda x: x.shape)

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_all_frequencies_all_windows" in col][0]].iloc[0].shape

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "lfp_timestamps" in col][0]].iloc[0]

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_timestamps" in col][0]].iloc[0]

- Checking if the right frequencies are being used

In [ ]:
LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_frequencies" in col]].head()

In [ ]:
LFP_TRACES_DF["granger_calculation_frequencies"] = LFP_TRACES_DF[[col for col in LFP_TRACES_DF.columns if "granger_frequencies" in col][0]].copy()

- Dropping unnecessary columns

In [ ]:
LFP_TRACES_DF = LFP_TRACES_DF.drop(columns=[col for col in LFP_TRACES_DF.columns if "granger_frequencies" in col], errors="ignore")

In [ ]:
LFP_TRACES_DF.head()

## Calculating the averages

In [ ]:
LFP_TRACES_DF.columns

In [ ]:
# LFP_TRACES_DF.to_pickle("./proc/rce2_spectral_granger.pkl")
LFP_TRACES_DF.to_pickle("./proc/rce_pilot_2_02_full_spectral.pkl")
# LFP_TRACES_DF.to_pickle("/blue/npadillacoreano/ryoi360/projects/reward_comp/final_proc/rce_pilot_2_02_spectral_granger.pkl")

In [ ]:
raise ValueError()